In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
colab = False

if colab:
    from google.colab import drive
    drive.mount('/content/drive')
    os.chdir('/content/drive/MyDrive/Tesis/tesis/classificador')
else:
    os.chdir('../classificador')

In [3]:
from datasets.speech_commands_dataset import *
from transforms import transforms_wav as twav
from transforms import transforms_stft as tstft
from torchvision.transforms import *

In [4]:
background_noise = 'datasets/_background_noise_'
n_mels = 32

In [5]:
data_aug_transform = Compose([
    twav.ChangeAmplitude(), 
    twav.ChangeSpeedAndPitchAudio(), 
    twav.FixAudioLength(), 
    tstft.ToSTFT(), 
    tstft.StretchAudioOnSTFT(), 
    tstft.TimeshiftAudioOnSTFT(), 
    tstft.FixSTFTDimension()])
bg_dataset = BackgroundNoiseDataset(background_noise, data_aug_transform)
add_bg_noise = tstft.AddBackgroundNoiseOnSTFT(bg_dataset)
train_feature_transform = Compose([
    tstft.ToMelSpectrogramFromSTFT(n_mels=n_mels), 
    tstft.DeleteSTFT(), 
    twav.ToTensor('mel_spectrogram', 'input')])

In [8]:
train_dataset = SpeechCommandsDataset('../data/classifier/train.csv',
                                Compose([twav.LoadAudio(),
                                         data_aug_transform,
                                         add_bg_noise,
                                         train_feature_transform]))

In [11]:
train_dataset.classes

['OTHER', 'BOAFAB']

In [16]:
import pandas as pd
pd.DataFrame(train_dataset.data)[1].value_counts(True)

1
1    0.505332
0    0.494668
Name: proportion, dtype: float64